# **Identifying differentially expressed genes involved in Tetralogy of Fallot**

Tetralogy of Fallot (TOF) is a congenital heart defect that affects the structure of the heart and its ability to pump blood to the body (1). TOF is one of the most common forms of congenital heart disease, accounting for around 10% of all cases (2). The condition is characterized by a combination of four abnormalities in the heart: a ventricular septal defect (VSD), narrowing of the pulmonary artery (pulmonary stenosis), an enlarged aorta that overrides the VSD, and right ventricular hypertrophy (thickening of the muscle in the right ventricle) (3). These abnormalities can cause oxygen-poor blood to be pumped to the body, resulting in symptoms such as cyanosis (bluish discoloration of the skin), shortness of breath, and fatigue. While TOF is a serious condition, advances in medical treatment have greatly improved the long-term outlook for those born with the defect (4).

Locating differentially expressed genes can provide valuable insights into the underlying biological mechanisms of Tetralogy of Fallot (TOF) and help identify potential therapeutic targets. Differentially expressed genes are genes that are expressed at different levels between affected individuals and healthy controls. These genes can be identified using techniques such as microarray analysis or RNA sequencing.

In today's assignment we will use a dataset from the EMBL-EBI's Expression Atlas to locate differentially expressed genes for TOF (5, 6). You will visualize your results with a volcano plot. Let's get started!

**First run the following code to import some libraries that we will need.**

In [ ]:
# importing libraries
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
from scipy.stats import ttest_ind

#### **Exercise 3.1**

First, we have to load our data, the file loaded below contains the following information:
- Experimental design
  - `Run`: ID for every separate patient.
  - `disease`: gives information if the patient was healthy (`normal`) or affected (`Tetralogy of Fallot`).
  - `Gene ID`: Separate identifier for every separate gene.
  - `Gene Name`
  - `Gene ID`
  - `Gene Name`: depicting all measured genes
  - Multiple columns starting with `SR...`: ID for every separate patient with ofcourse the corresponding gene count.

Both files contain more information that we will not use for this assignment, but you can have a look if you are interested :)

In order to work with the data we have to combine the two files.

Run the following code block to load the important data, and have a look at the what information is stored there.

*hint: Look at the documentation for the suggested functions to know how to use them.*

In [ ]:
# 1. Reading files
df_ = pd.read_csv('normalized_counts.csv')
design = pd.read_csv('experiment_design.txt', sep='\t')[['Run', 'disease']]

# 2. Transforming file into long format
df_ = df_.melt(id_vars=['Gene ID', 'Gene Name'], var_name='Run', value_name='geneCount')

# 3. and 4. Merging and selection of columns
df = df_.merge(design, on='Run')[['Gene ID', 'Gene Name', 'Run', 'geneCount', 'disease']]
df

#### **Exercise 3.2**

Now we have to clean up the data a bit and filter out low gene counts. We do the following:

Remove the genes that have a **total sum** gene count under 100.000

If you run the following code it will return an error of a missing variable. Locate the missing variable and assign the correct value.

In [ ]:
# Insert missing variable here:

# 2. Remove genes with low gene count
sel = df.groupby('Gene Name', as_index=False).sum()
sel = sel[sel.geneCount >= gene_count_threshold]['Gene Name']
df = df[df['Gene Name'].isin(sel)]
len(df)

#### **Exercise 3.3**

Complete the missing parts in the following code block. The function is supposed to return the following two measures:

Complete the code after the comment blocks for:

1. Calculate the foldchange by taking the mean of `tof` and dividing it by the mean of `norm`. Store this into a new variable `fc`. *Hint: you can use the function `np.mean()` if you want to calculate the average.

2. Before you can store your foldchange and p-value you need to transform both as follows:
  - $log_{2} (FoldChange)$
  - $-log_{10} (P value)$
 
 the p-value is already transformed, do the same for the foldchange (`fc`)


*Hint: You can use the functions `np.log2` and `np.log10` for the log transforms*

In [ ]:
def p_val(x):

    # Selecting the both disease conditions
    tof = x[x.disease == 'Tetralogy of Fallot']['geneCount']
    norm = x[x.disease == 'normal']['geneCount']

    ## INSERT CODE HERE (1):
    fc = 

    # Calculate the p_value
    z, p = ttest_ind(norm, tof)

    # transform p-value
    x['pValue'] = -np.log10(p)
    
    ## INSERT CODE HERE (2):
    x['foldChange'] = 
    
    # Return your new results
    return x

The next step is to apply the function to the dataframe, we want to calculate the p-value and foldchange for every separate gene which we do with a very convenient function called `groupby()`. If you are interested you can look up the documentation for this function.

Run the code below.

In [ ]:
df = df.groupby('Gene ID', as_index=False).apply(lambda x: p_val(x))

#### **Exercise 3.5**

Now we come to the most fun part, plotting our results! In the following line you can see two parameters:

- `fc_b`: this depicts the cut-off value for the $log_{2}(foldchange)$. This means that you will only see the Gene names appear that are either smaller than `-fc_b` or larger than `fc_b`.
- `alpha`: The probability of incorrectly rejecting a true null hypothesis. We want our p-value to be smaller to be able to say that our two conditions are significantly different. By convention the value of 0.05 is most used for the alpha.

Play around with both parameters and run both blocks of code to see what happens.

When you have a nice collection of genes try to write a small report if the found genes make sense regarding the disease. (hint: look up the gene names online and see if you can find anything about their function)

In [ ]:
fc_b = 
alpha = 

In [ ]:
# Initialize figure
plt.figure(figsize=(15,10))

# log transform the alpha value
p_b = -np.log10(alpha)

# Start with plotting all points
plt.scatter(x=df['foldChange'], y=df['pValue'], s=10, label="Not significant")

# highlight down- or up- regulated genes
down = df[(df['foldChange'] <= -fc_b) & (df['pValue'] >= p_b)]
up = df[(df['foldChange'] >= fc_b) & (df['pValue'] >= p_b)]
plt.scatter(x=down['foldChange'], y=down['pValue'],s=3, label="Down-regulated",color="blue")
plt.scatter(x=up['foldChange'], y=up['pValue'], s=3, label="Up-regulated",color="red")

# Add text to highlighted genes
for i, r in up.iterrows():
    plt.text(x=r['foldChange'], y=r['pValue'], s=r['Gene Name'], fontsize=12)

for i, r in down.iterrows():
    plt.text(x=r['foldChange'], y=r['pValue'], s=r['Gene Name'], fontsize=12)

# Plot layout
plt.xlabel("log2 FC", fontsize=20)
plt.ylabel("-log10 p-value", fontsize=20)
plt.axvline(-fc_b, color="grey",linestyle="--")
plt.axvline(fc_b, color="grey",linestyle="--")
plt.axhline(p_b, color="grey",linestyle="--")
plt.legend(fontsize=20)

References:

1. American Heart Association. (2022). Tetralogy of Fallot. Retrieved from https://www.heart.org/en/health-topics/congenital-heart-defects/about-congenital-heart-defects/tetralogy-of-fallot
2. Centers for Disease Control and Prevention. (2022). Facts about Tetralogy of Fallot. Retrieved from https://www.cdc.gov/ncbddd/heartdefects/tetralogyoffallot.html
4. Mayo Clinic. (2022). Tetralogy of Fallot. Retrieved from https://www.mayoclinic.org/diseases-conditions/tetralogy-of-fallot/symptoms-causes/syc-20353477
4. Mocarski, M., & Bilek, L. (2021). Tetralogy of Fallot. StatPearls [Internet]. Retrieved from https://www.ncbi.nlm.nih.gov/books/NBK513263/
5. Expression Atlas (EMBI-EBI). Retrieved from https://www.ebi.ac.uk/gxa/home
6. Grunert M, Dorn C, Schueler M, Dunkel I, Schlesinger J et al. (2014) Rare and private variations in neural crest, apoptosis and sarcomere genes define the polygenic background of isolated Tetralogy of Fallot.
